In [1]:
import pandas as pd
import numpy as np
import re

# Path
path_raw = r'C:/Users/Usuario/Documents/b4d/data/raw/'
path_processed = r'C:/Users/Usuario/Documents/b4d/data/processed/'

# Name of file
file_name = 'terbol_clientes_número_ejecutivos_1'
ext = '.xlsx'

file_name_2 = 'terbol_clientes_numeros_ejecutivo_new'

file_name_1 = 'terbol_SalesExecutive-2025-06-10'
ext_1 = '.csv'

file_brch = 'terbol_TerbolSucursal-2025-06-10'

suffix = '_clean'

In [6]:
import pandas as pd

df_antiguo = pd.read_excel(path_raw + 'terbol_clientes_número_ejecutivos_1.xlsx')
df_nuevo = pd.read_excel(path_raw + 'terbol_clientes_numeros_ejecutivo_new.xlsx')

clave_columnas = ['IDCLI', 'IDSUC.', 'CLIENTE', 'LABORATORIO']

if not all(col in df_antiguo.columns and col in df_nuevo.columns for col in clave_columnas):
    raise ValueError("Alguna columna de la clave no existe en los DataFrames.")

nuevos_registros = df_nuevo[~df_nuevo[clave_columnas].apply(tuple, axis=1).isin(
    df_antiguo[clave_columnas].apply(tuple, axis=1))]

comunes = pd.merge(df_antiguo, df_nuevo, on=clave_columnas, suffixes=('_antiguo', '_nuevo'))
cambios_detallados = []

for index, row in comunes.iterrows():
    diferencias = {}
    for col in [c for c in df_antiguo.columns if c not in clave_columnas]:
        if pd.isna(row[f'{col}_antiguo']) and pd.isna(row[f'{col}_nuevo']):
            continue
        if row[f'{col}_antiguo'] != row[f'{col}_nuevo']:
            diferencias[col] = {'antiguo': row[f'{col}_antiguo'], 'nuevo': row[f'{col}_nuevo']}
    if diferencias:
        for col in clave_columnas:
            diferencias[col] = row[col]
        cambios_detallados.append(diferencias)

with pd.ExcelWriter('resultados_comparacion.xlsx') as writer:
    if not nuevos_registros.empty:
        nuevos_registros.to_excel(writer, sheet_name='Nuevos_Registros', index=False)
    if cambios_detallados:
        pd.DataFrame(cambios_detallados).to_excel(writer, sheet_name='Registros_Cambiados', index=False)

print(f"Se han identificado {len(nuevos_registros)} nuevos registros.")
print(f"Se han identificado {len(cambios_detallados)} registros con cambios.")

Se han identificado 304 nuevos registros.
Se han identificado 5958 registros con cambios.


In [2]:
df = pd.read_excel(path_raw+file_name+ext)

df_2 = pd.read_excel(path_raw+file_name_2+ext)


In [3]:
df.columns

Index(['GERENCIA', 'SUPERVISOR', 'VENDEDOR', 'IDCLI', 'IDSUC.', 'CLIENTE',
       'LABORATORIO', 'NUMERO VENDEDOR', 'CORREO ELECTRONICO'],
      dtype='object')

In [ ]:

df_se = df.copy()


In [ ]:
df_se['type'] = np.where(df_se['VENDEDOR'].str.contains('EXTERNO', na=False), 'EXTERNO', None)
df_se['sales_manager_type'] = np.where(df_se['SUPERVISOR'].str.contains('EXTERNO', na=False), 'EXTERNO', None)

df_se['NUMERO VENDEDOR'] = df_se['NUMERO VENDEDOR'].astype(str)
df_se['CORREO ELECTRONICO'] = df_se['CORREO ELECTRONICO'].astype(str)

df_se['NUMERO VENDEDOR'] = df_se['NUMERO VENDEDOR'].str.replace(' ','')

In [ ]:
regex_1 = r'\+591'
regex_2 = r'\- EXTERNO'

def f_regex(data_str, regex, str_replace=''):
    return re.sub(regex, str_replace, data_str, flags=re.IGNORECASE)


df_se['phone_number'] = df_se['NUMERO VENDEDOR'].apply(lambda x: f_regex(x, regex_1))
df_se['name'] = df_se['VENDEDOR'].apply(lambda x: f_regex(x, regex_2))
df_se['sales_manager'] = df_se['SUPERVISOR'].apply(lambda x: f_regex(x, regex_2))

In [34]:
df_se['phone_number'] = df_se['phone_number'].str.strip()
df_se['name'] = df_se['name'].str.strip()
df_se['sales_manager'] = df_se['sales_manager'].str. strip()

In [35]:
# df_se['sales_manager_office'] = df_se['GERENCIA'].str.capitalize()
df_se['phone_number'] = np.where(df_se['phone_number'] == '0', None, df_se['phone_number'])
df_se['CORREO ELECTRONICO'] = np.where(df_se['CORREO ELECTRONICO'].str.strip() == '0', None, df_se['CORREO ELECTRONICO'])

In [36]:
df_se.rename(columns={'CORREO ELECTRONICO': 'email', 'GERENCIA': 'sales_manager_office'}, inplace=True)
df_se = df_se[['name' , 'phone_number', 'email', 'type', 'sales_manager', 'sales_manager_type', 'sales_manager_office', ]]

In [37]:
df_se.drop_duplicates(inplace=True)

In [38]:

df_se['created_data'] = pd.Timestamp.now()
df_se['updated_data'] = pd.Timestamp.now()
df_se['updated_data'] = df_se['updated_data'].dt.strftime('%d/%m/%Y %H:%M')
df_se['created_data'] = df_se['created_data'].dt.strftime('%d/%m/%Y %H:%M')

In [ ]:
df_se.to_excel(path_processed+'terbol_clientes_número_ejecutivos_1' + suffix+'.xlsx', index=False)

In [ ]:
result = df.groupby('CLIENTE', as_index=False)['VENDEDOR'].nunique()
result

In [ ]:
df_sucursal_new = df.copy()
df_sucursal_new.columns

In [ ]:
df_sucursal_new = df_sucursal_new[['IDCLI', 'IDSUC.', 'CLIENTE']]

In [ ]:
df_sucursal_new['razon_social'] = np.where(
    df_sucursal_new['CLIENTE'].str.split(' - ').str[0].str.contains('FARMACIA'),
    None, 
    df_sucursal_new['CLIENTE'].str.split(' - ').str[0]
)
df_sucursal_new['nombre_comercial'] = np.where(
    df_sucursal_new['CLIENTE'].str.split(' - ').str[1].isna(), 
    np.where(
        df_sucursal_new['CLIENTE'].str.split(' - ').str[0].str.contains('FARMACIA'),
        df_sucursal_new['CLIENTE'].str.split(' - ').str[0],
        None
    ),
    df_sucursal_new['CLIENTE'].str.split(' - ').str[1]
)

In [ ]:
df_sucursal_new['gestion_megalabs'] = False
df_sucursal_new['gestion_tecnofarma'] = False
df_sucursal_new['created_data'] = pd.Timestamp.now()
df_sucursal_new['updated_data'] = pd.Timestamp.now()

In [ ]:
df_sucursal_new.columns

In [ ]:
df_sucursal_new.rename(columns={'IDCLI':'cod_cliente', 'IDSUC.':'cod_sucursal'}, inplace=True)

In [ ]:
df_sucursal_new = df_sucursal_new[['cod_cliente', 'cod_sucursal', 'razon_social', 'nombre_comercial',
       'gestion_megalabs', 'gestion_tecnofarma', 'created_data',
       'updated_data']]

In [ ]:
df_sucursal_new.drop_duplicates(inplace=True)

In [ ]:

df_branch = pd.read_csv(path_raw + file_brch + ext_1)
df_sucursal_new = df_sucursal_new.merge(df_branch, right_on=['cod_cliente', 'cod_sucursal'], left_on=['cod_cliente', 'cod_sucursal'], how='left', suffixes=('','_b'))
df_sucursal_new = df_sucursal_new[df_sucursal_new['id'].isna()]


In [ ]:
df_sucursal_new = df_sucursal_new[['cod_cliente', 'cod_sucursal', 'razon_social', 'nombre_comercial',
       'gestion_megalabs', 'gestion_tecnofarma', 'created_data',
       'updated_data']]

In [ ]:
df_sucursal_new.to_csv(path_processed + 'new_sucursales' + suffix + ext_1, index=False)

##
##

In [60]:
df_bse = df.copy()
df_bse.columns

Index(['GERENCIA', 'SUPERVISOR', 'VENDEDOR', 'IDCLI', 'IDSUC.', 'CLIENTE',
       'LABORATORIO', 'NUMERO VENDEDOR', 'CORREO ELECTRONICO'],
      dtype='object')

In [61]:
# 
df_se_id = pd.read_csv(path_raw + file_name_1 + ext_1)
df_branch = pd.read_csv(path_raw + file_brch + ext_1)


In [62]:
df_branch.columns

Index(['id', 'cod_cliente', 'cod_sucursal', 'nit', 'razon_social',
       'nombre_comercial', 'direccion', 'oficina', 'municipio', 'provincia',
       'canal', 'gerencia', 'cond_pago_mlb', 'atencion_mlb',
       'gestion_megalabs', 'gestion_tecnofarma', 'created_data',
       'updated_data'],
      dtype='object')

In [63]:
df_bse = df_bse.merge(df_branch, left_on=['IDCLI','IDSUC.'], right_on=['cod_cliente', 'cod_sucursal'], how='left')
df_bse.shape

(13606, 27)

In [64]:
# separamos los sucursal terbol que no tiene id 
df_bse = df_bse[df_bse['id'].notna()]


In [65]:
df_bse.shape

(13606, 27)

In [66]:

regex_2 = r'\- EXTERNO'

def f_regex(data_str, regex, str_replace=''):
    return re.sub(regex, str_replace, data_str, flags=re.IGNORECASE)

df_bse['name_cleaned'] = df_bse['VENDEDOR'].apply(lambda x: f_regex(x, regex_2))
df_bse['name_cleaned'] = df_bse['name_cleaned'].str.strip()

In [67]:
df_bse = df_bse.merge(df_se_id, left_on='name_cleaned', right_on='name', how='left')
df_bse.shape

(13606, 38)

In [47]:
df_bse.columns

Index(['GERENCIA', 'SUPERVISOR', 'VENDEDOR', 'IDCLI', 'IDSUC.', 'CLIENTE',
       'LABORATORIO', 'NUMERO VENDEDOR', 'CORREO ELECTRONICO', 'id_x',
       'cod_cliente', 'cod_sucursal', 'nit', 'razon_social',
       'nombre_comercial', 'direccion', 'oficina', 'municipio', 'provincia',
       'canal', 'gerencia', 'cond_pago_mlb', 'atencion_mlb',
       'gestion_megalabs', 'gestion_tecnofarma', 'created_data',
       'updated_data', 'name_cleaned', 'id_y', 'name', 'phone_number', 'email',
       'type', 'sales_manager', 'sales_manager_type', 'sales_manager_office',
       'created_date', 'updated_date'],
      dtype='object')

In [68]:
df_bse['created_data'] = pd.Timestamp.now(tz="UTC")
df_bse['updated_data'] = pd.Timestamp.now(tz="UTC")


df_bse['created_data'] = df_bse['created_data'].dt.strftime('%d/%m/%Y %H:%M')
df_bse['updated_data'] = df_bse['updated_data'].dt.strftime('%d/%m/%Y %H:%M')

In [ ]:
df_bse.info()

In [69]:

df_bse.rename(columns={'id_x': 'terbol_sucursal', 'id_y': 'sales_executive', 'cod_sucursal':'branch_cod', 'LABORATORIO': 'laboratory_name'}, inplace=True)

In [70]:
df_bse = df_bse[['branch_cod', 'laboratory_name', 'created_data', 'updated_data', 'sales_executive', 'terbol_sucursal']]

In [72]:
df_bse['terbol_sucursal'] = df_bse['terbol_sucursal'].astype(int)
df_bse['sales_executive'] = df_bse['sales_executive'].astype(int) 
df_bse['branch_cod'] = df_bse['branch_cod'].astype(int) 

In [73]:
df_bse_1 = df_bse[:1]
df_bse_1

,branch_cod,laboratory_name,created_data,updated_data,sales_executive,terbol_sucursal
0,100004110,MEGALABS,10/06/2025 13:38,10/06/2025 13:38,65,9823


In [75]:
df_bse.to_excel(path_processed + 'new_branch_sucursales_prod' + suffix + ext, index=False)

# df_bse_1.to_csv(path_processed + 'new_branch_sucursales_prod' + suffix + ext_1, index=False)

In [ ]:
result = result[result['VENDEDOR']>1]
result

In [ ]:
# Un Id sucursal no tiene mas de un cliente
result = df.groupby('IDSUC.', as_index=False)['CLIENTE'].nunique()
result = result[result['CLIENTE']>1]
result

In [ ]:
# No existe sucursal lab asignado mas de un vendedor
result = df.groupby(['IDSUC.', 'LABORATORIO'], as_index=False)['VENDEDOR'].nunique()
result = result[result['VENDEDOR']>1]
result

In [ ]:
# No existe sucursal lab asignado mas de un vendedor
result = df.groupby(['VENDEDOR', 'LABORATORIO'], as_index=False).nunique()
# result = result[result['VENDEDOR']>1]
result